# 1. Install Dependencies

pip install torch from https://pytorch.org/ based on your specs

In [1]:
#install YOLOv5
!git clone https://github.com/ultralytics/yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [2]:
!cd yolov5 & pip install -r requirements.txt

# 2. Realtime Gauze Detection w/ 2 Webcams

In [2]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import time

# YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'custom', path='../yolov5-master/202318JUL.pt', force_reload=True)
model2 = torch.hub.load('ultralytics/yolov5', 'yolov5x6', force_reload=True)
model.to("cuda")
model2.to("cuda")

model.conf = 0.50
#model.max_det = 20
model.classes = [0]
model2.conf = 0.25
#model2.max_det = 20
model2.classes = [0]

onscreenIn, onscreenOut, countIn, countOut, frameCountIn, frameCountOut, startTime, endTime, countPlay = 0, 0, 0, 0, 0, 0, 0, 0, 0
condition = 7
cap1 = cv2.VideoCapture(0)
cap2 = cv2.VideoCapture(1)
ret, frame1 = cap1.read()
ret, frame2 = cap2.read()

pic1=frame1.shape[1]
pic2=frame2.shape[1]
start_time = cv2.getTickCount()


while True:
    ret, frame1 = cap1.read()
    ret, frame2 = cap2.read()
    
    
    # Make detections
    pic = np.hstack((frame1, frame2))
    pic_gpu=torch.tensor(pic,device="cuda")
    results = model(pic)
    results2 = model2(pic)

    if len(results.xyxy) > 0:
        detections = results.xyxy[0]
        gauze_indices = torch.nonzero(detections[:, 5].long() == model.classes[0]).squeeze()
        xmax_indices = detections[:, 2]

        a = (xmax_indices[gauze_indices] <= pic1).sum().item()
        b = (xmax_indices[gauze_indices] > pic1).sum().item()

    if len(results2.xyxy) > 0:
        detections2 = results2.xyxy[0]
        human_indices = torch.nonzero(detections2[:, 5].long() == 0).squeeze()
        xmax_indices2 = detections2[:, 2]

        c = (xmax_indices2[human_indices] <= pic1).sum().item()
        d = (xmax_indices2[human_indices] > pic1).sum().item()
    
    image = np.squeeze(results.render())
    borderedFrame = cv2.copyMakeBorder(image, 0, 350, 640 - pic1, 640 - pic2, cv2.BORDER_CONSTANT,
                                       value=[0, 0, 0])

    if c == 0:
        tempcount1 = a
        if tempcount1 != onscreenIn:
            frameCountIn += 1
            if frameCountIn == condition:
                if tempcount1 > onscreenIn:
                    countIn += tempcount1 - onscreenIn
                onscreenIn = tempcount1
        else:
            frameCountIn = 0
        cv2.putText(borderedFrame, 'On Screen =' + str(onscreenIn), (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(borderedFrame, 'Total In =' + str(countIn), (300, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255),
                    2, cv2.LINE_AA)
    else:
        cv2.putText(borderedFrame, 'On Screen =' + 'Paused', (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)
        cv2.putText(borderedFrame, 'Total In =' + 'Paused', (300, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)
        cv2.putText(borderedFrame, 'Human Detected', (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)

    if d == 0:
        tempcount2 = b
        if tempcount2 != onscreenOut:
            frameCountOut += 1
            if frameCountOut == condition:
                if tempcount2 > onscreenOut:
                    countOut += tempcount2 - onscreenOut
                onscreenOut = tempcount2
        else:
            frameCountOut = 0
        cv2.putText(borderedFrame, 'On Screen =' + str(onscreenOut), (940, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)
        cv2.putText(borderedFrame, 'Total Out =' + str(countOut), (940, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)
    else:
        cv2.putText(borderedFrame, 'On Screen =' + 'Paused', (940, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(borderedFrame, 'Total Out =' + 'Paused', (940, 55), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(borderedFrame, 'Human Detected', (840, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # FPS Counter
    end_time = cv2.getTickCount()
    elapsed_time = (end_time - start_time) / cv2.getTickFrequency()
    fps = 1 / elapsed_time
    start_time = end_time
    cv2.putText(borderedFrame, "FPS =" + str(round(fps, 1)), (540, 520), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255),
                2, cv2.LINE_AA)

    # Gauze In-Play Counter
    countPlay = countIn - onscreenIn - countOut
    #cv2.putText(borderedFrame, 'In Play =' + str(countPlay), (360, 650), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 0, 255), 4,
    #            cv2.LINE_AA)

    cv2.imshow('Gauze', borderedFrame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap1.release()
cap2.release()
cv2.destroyAllWindows()

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\user/.cache\torch\hub\master.zip
YOLOv5  2023-8-10 Python-3.7.16 torch-1.7.1+cu110 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)

Fusing layers... 
Model summary: 267 layers, 46108278 parameters, 0 gradients, 107.6 GFLOPs
Adding AutoShape... 
Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\user/.cache\torch\hub\master.zip
YOLOv5  2023-8-10 Python-3.7.16 torch-1.7.1+cu110 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)

Fusing layers... 
YOLOv5x6 summary: 574 layers, 140730220 parameters, 0 gradients, 209.6 GFLOPs
Adding AutoShape... 
